In [ ]:
import boto3
from botocore.exceptions import ClientError
from types_boto3_dynamodb.service_resource import Table
dynamodb=boto3.resource("dynamodb")
resource:Table=dynamodb.Table("practice")

put item

In [ ]:
try:
    response=resource.put_item(
        Item={
            "pk":"2",
            "sk":"details"
        }
    )
    
except  ClientError as e:
    print(type(e))
    print(str(e))

Get Item

In [ ]:
response=resource.get_item(
    Key={
        "pk":"1",
        "sk":"details"
    }
)
print(response)

In [ ]:
response.get("Item")

query item

In [ ]:
from boto3.dynamodb.conditions import Key
response=resource.query(
    KeyConditionExpression=Key("pk").eq("1") & Key("sk").begins_with("PROF"),
    
)
print(response)

In [ ]:
print(response.get("Items"))

query with partiql

In [ ]:

client = resource.meta.client

stmt = "SELECT pk, sk, name FROM \"practice\" WHERE pk='1' AND begins_with(sk, 'PROF')"

partiql_resp = client.execute_statement(
    Statement=stmt
)
print(partiql_resp)

In [ ]:

partiql_resp.get("Items")

Batch get

In [ ]:
client = resource.meta.client

request_items = {
    "practice": {
        "Keys": [
            {"pk":'1', "sk": "details"},
            {"pk": "2", "sk":  "details"},
        ],
    }
}

resp = client.batch_get_item(RequestItems=request_items)
print(resp)

In [ ]:
items = resp.get("Responses", {}).get("practice", [])

while resp.get("UnprocessedKeys"):
    resp = client.batch_get_item(RequestItems=resp["UnprocessedKeys"])
    items.extend(resp.get("Responses", {}).get("practice", []))

items

batch get partiql


In [ ]:
statements=[
    {
        "Statement":"Select * FROM \"practice\" where pk='1' and sk='details'"
    },
    {
        "Statement":"Select * FROM \"practice\" where pk='2' and sk='details'"
    }
    
]
resp=client.batch_execute_statement(Statements=statements)
print(resp)

In [ ]:
items = resp.get("Responses", {})
for item in items:
    print(item.get("Item"))

batch write

In [ ]:
client = resource.meta.client

response = client.batch_write_item(
    RequestItems={
        "practice": [
            {
                "PutRequest": {
                    "Item": {
                        "pk": "3",
                        "sk": "details",
                        "name":"ganesh"
                    }
                }
            },
            {
                "PutRequest": {
                    "Item": {
                        "pk": "4",
                        "sk": "details",
                        "name":"suresh"
                    }
                }
            },
        ]
    }
)

print(response)



partiql batchwrite

In [ ]:
client = resource.meta.client

statements = [
    {
        "Statement": "INSERT INTO \"practice\" VALUE {'pk':'8','sk':'details','name':'rahul'}"
    },
    {
        "Statement":"INSERT INTO \"practice\" VALUE {'pk':'9','sk':'details','name':'roshan'}"
    }
]

resp = client.batch_execute_statement(Statements=statements)
print(resp)

conditional expression with put

In [ ]:
try:
    item={
        "pk":"1",
        "sk":"details"
    }

    resource.put_item(
        Item=item,
        ConditionExpression="attribute_not_exists(pk)"
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'ConditionalCheckFailedException':
        print("Item already exists, not overwritten.")
    else:
        raise e

transaction

In [ ]:
client = resource.meta.client
try:
    client.transact_write_items(
        TransactItems=[
            {
                "Put": {
                    "TableName": "practice",
                    "Item": {
                        "pk": "10",
                        "sk": "details",
                    },
                }
            },
            {
                "Put": {
                    "TableName": "practice",
                    "Item": {
                        "pk": "10",
                        "sk": "PROFILE",
                    },
                }
            },
        ]
    )
except ClientError as err:
    raise


update item

In [ ]:
try:
    response=resource.update_item(
        Key={
            "pk":"1",
            "sk":"details"
        },
        UpdateExpression='SET #att1=:val1',
        ExpressionAttributeNames={
            '#att1':'name'
        },
        ExpressionAttributeValues={
            ':val1':'ne zha',
        },
        ReturnValues="ALL_NEW",  
        ConditionExpression="attribute_exists(pk)"
    )
    
    print(response)
except ClientError as e:
    print(e)


transact update item

In [ ]:
client=resource.meta.client
response = client.transact_write_items(
                TransactItems=[
                    {
                        "Update": {
                            "TableName": "practice",
                            "Key": {"pk": "1","sk":"details"},
                            "UpdateExpression": "SET #attr_name=:new_value",
                            "ExpressionAttributeNames": {"#attr_name": "name"},
                            "ExpressionAttributeValues": {":new_value": "new_name"},
                            "ConditionExpression": "attribute_exists(pk)",
                        }
                    },
                    {
                        "Update": {
                            "TableName": "practice",
                            "Key": {"pk": "2","sk":"details"},
                            "UpdateExpression": "SET #is_blocked=:new_value",
                            "ExpressionAttributeNames": {"#attr_name": "name"},
                            "ExpressionAttributeValues": {":new_value": "new_name2"},
                            "ConditionExpression": "attribute_exists(pk)",
                        }
                    },
                ]
            )
print(response)